In [1]:
import pandas as pd
import numpy as np
import sklearn as sk

In [2]:
df = pd.read_csv('resume.csv')

In [3]:
df.shape

(2369, 20)

In [4]:
df.duplicated().shape

(2369,)

In [5]:
df.head(1)

,Unnamed: 0,age,auto,business_trips,categories,citizenship,city,education,employment,experience,experience_length,family,gender,languages,metro_or_moving,name,photo_url,salary,skills,url
0,0,20,NaN,не готов к командировкам,Архитектура,Россия,Ковров,"[{'kind': 'Неполное высшее образование', 'form...",неполный рабочий день,"[{'period': 'сентябрь 2017 – март 2018', 'leng...",7 месяцев,NaN,f,"['Немецкий язык, Базовый']",NaN,"3D-моделлер, IT стажёр, консультант",NaN,30 000₽,NaN,https://www.superjob.ru/resume/3d-modeller-436...


In [6]:
df.columns

Index(['Unnamed: 0', 'age', 'auto', 'business_trips', 'categories',
       'citizenship', 'city', 'education', 'employment', 'experience',
       'experience_length', 'family', 'gender', 'languages', 'metro_or_moving',
       'name', 'photo_url', 'salary', 'skills', 'url'],
      dtype='object')

In [7]:
df = df.drop('Unnamed: 0', axis=1)

In [8]:
df.auto.value_counts(dropna=False)

['B — легковые авто']                                                                                 1171
NaN                                                                                                    808
['B — легковые авто', 'C — грузовые авто']                                                             220
['A — мотоциклы', 'B — легковые авто']                                                                  74
['A — мотоциклы', 'B — легковые авто', 'C — грузовые авто']                                             52
['C — грузовые авто']                                                                                   11
['B — легковые авто', 'C — грузовые авто', 'D — автобусы']                                              11
['B — легковые авто', 'C — грузовые авто', 'D — автобусы', 'E — авто с прицепом']                        7
['B — легковые авто', 'D — автобусы']                                                                    5
['A — мотоциклы', 'B — легковые авто'

Много NaN, выкидываем.

In [9]:
df = df.drop('auto', axis=1)

In [10]:
df.business_trips.value_counts(dropna=False)

NaN                         1373
не готов к командировкам     968
на дому                       28
Name: business_trips, dtype: int64

Много NaN, выкидываем.

In [11]:
df = df.drop('business_trips', axis=1)

In [12]:
df.categories.value_counts(dropna=False)

Системное администрирование                      552
NaN                                              467
Техническая поддержка                            358
IT, Интернет, связь, телеком                     238
Инжиниринг                                       184
Администрирование баз данных                      80
Административная работа, секретариат, АХО         64
Сетевые технологии                                49
Разработка, программирование                      41
Передача данных и доступ в интернет               38
Управление проектами                              34
Аналитика                                         31
Интернет, создание и поддержка сайтов             23
Другое                                            18
Электронный документооборот                       15
Внедрение и сопровождение ПО                      12
Делопроизводство, ввод данных, систематизация     11
Компьютеры, оргтехника, ПО                         9
Call Center                                   

Выкинуть незаполненные (20 % данных) или весь признак? Неясно

In [13]:
df = df.drop('categories', axis=1)

Так как категории заполнялись сайтом автоматически, эту информацию можно извлечь из других полей (name), удаляем.

In [14]:
df.citizenship.value_counts(dropna=False)

Россия               1615
NaN                   727
Украина                 8
Казахстан               7
Азербайджан             3
Кыргызстан              3
Беларусь                2
Дальнее зарубежье       1
Молдова                 1
Таджикистан             1
Литва                   1
Name: citizenship, dtype: int64

Считаем, что гражданство не влияет на уровень зп.

In [15]:
#df = df[(df.citizenship == 'Россия')  | df.citizenship.isna()]
df = df.drop('citizenship', axis=1)

In [16]:
df.city.value_counts(dropna=False)

Москва                                 720
Санкт-Петербург                        293
Хабаровск                               64
Уфа                                     55
Иркутск                                 51
Ростов-на-Дону                          48
Краснодар                               33
Самара                                  32
Саратов                                 27
Казань                                  27
Рязань                                  26
Нижний Новгород                         24
Ульяновск                               23
Пенза                                   23
Волгоград                               22
Воронеж                                 20
Оренбург                                19
Сыктывкар                               17
Тверь                                   17
Владимир                                16
Смоленск                                16
Челябинск                               16
Екатеринбург                            16
Тюмень     

Обучать модель для регионов отдельно не хочется, будет слишком мало объектов. Предложение: выделить категории в зависимости от уровня зарплат по региону. Для начала 3: Мск, Спб, остальные.

In [17]:
ctgr = pd.get_dummies(df.city)
city_Msk = ctgr['Москва']
city_Spb = ctgr['Санкт-Петербург']
ctgr = ctgr.drop(['Москва', 'Санкт-Петербург'], axis=1)
city_oth = np.sum(ctgr.values, axis=1)
df['city_Msk'] = city_Msk
df['city_Spb'] = city_Spb
df['city_oth'] = city_oth
df = df.drop(['city'], axis=1)
df.shape

(2369, 17)

In [18]:
df.employment.value_counts(dropna=False)

полная занятость           1180
NaN                        1107
неполный рабочий день        33
сменный график работы        25
частичная занятость          13
работа вахтовым методом      11
Name: employment, dtype: int64

Много NaN, выкидываем.

In [19]:
df = df.drop(['employment'], axis=1)

In [20]:
df.family.value_counts(dropna=False)

cостоит в браке, есть дети       804
NaN                              687
не состоит в браке, детей нет    528
cостоит в браке, детей нет       168
не состоит в браке, есть дети    111
cостоит в браке                   37
не состоит в браке                29
есть дети                          3
детей нет                          2
Name: family, dtype: int64

In [21]:
df = df.drop('family', axis=1)

In [22]:
gender = pd.get_dummies(df.gender)
gender.f
df['gender'] = gender.f

Как вариант можно считать, что если иностранные языки не указаны, то человек ими не владеет. Логика в этом есть!!!

In [23]:
df.languages.value_counts(dropna=False)

['Английский язык, Технический']                                                                             832
NaN                                                                                                          547
['Английский язык, Базовый']                                                                                 463
['Английский язык, Разговорный']                                                                             164
['Английский язык, Технический', 'Немецкий язык, Базовый']                                                    47
['Английский язык, Базовый', 'Немецкий язык, Базовый']                                                        42
['Английский язык, Cвободное владение']                                                                       40
['Английский язык, Технический', 'Русский язык, Cвободное владение']                                          16
['Английский язык, Технический', 'Немецкий язык, Технический']                                  

In [24]:
#df = df.drop('languages', axis=1)

In [25]:
df.metro_or_moving.value_counts(dropna=False)

NaN                                            1357
готов к переезду                                540
Пражская                                         12
Улица Дыбенко                                     9
Комендантский проспект                            8
Коломенская                                       6
Медведково                                        6
Выхино                                            6
Ботанический садВДНХ                              6
Проспект Ветеранов                                6
Первомайская                                      6
Ладожская                                         6
Новогиреево                                       6
Планерная                                         5
Проспект Большевиков                              5
Академическая                                     5
Щелковская                                        5
Новокосино                                        5
Войковская                                        4
Московская  

In [26]:
df = df.drop('metro_or_moving', axis=1)

In [27]:
df.name.value_counts(dropna=False)

Системный администратор, IT-специалист                                                                                                 416
IT-специалист                                                                                                                          207
IT специалист                                                                                                                           76
ИТ-специалист                                                                                                                           54
Руководитель IT-отдела                                                                                                                  37
Инженер IT                                                                                                                              29
Руководитель отдела ИТ                                                                                                                  27
Руководитель IT отдела     

In [28]:
df.name.unique().shape

(1115,)

Нужно обрабатывать как текст. Очень много синонимов!

In [29]:
df = df.drop('photo_url', axis=1)

In [30]:
df.skills.value_counts(dropna=False).shape

(1965,)

Часть - дубликаты. Но полных дубликатов не было. Видимо, люди хотят на разные должности по одинаковым резюме. Можно оставить как есть (логике задачи это не противоречит)

In [31]:
df.groupby(by='skills')

In [32]:
df[df.skills.isna()].shape

(330, 13)

In [33]:
df = df.dropna(subset=['skills'])

Видимо, придется выкинуть незаполненные, важная инфа :(

In [34]:
df = df.drop('url', axis = 1)

In [35]:
df.shape

(2039, 12)

In [36]:
df.education.value_counts(dropna=False)

NaN                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    43
[{'kind': 'Среднее образование'}]                                                                                                                                                             

In [37]:
#df = df[df.education.isna() == False]
df = df.dropna(subset=['education'])

Выкидываем незаполненные. Че дальше - непонятно

In [38]:
df.experience_length.value_counts(dropna=False)

NaN                     78
14 лет и 4 месяца       17
7 лет и 3 месяца        17
10 лет и 6 месяцев      14
9 лет и 7 месяцев       13
3 года и 6 месяцев      13
11 лет и 4 месяца       13
10 лет и 2 месяца       13
12 лет и 8 месяцев      13
11 лет и 11 месяцев     13
13 лет и 11 месяцев     12
7 лет и 2 месяца        12
5 лет и 10 месяцев      11
7 лет и 9 месяцев       11
9 лет и 5 месяцев       11
12 лет и 11 месяцев     11
15 лет и 2 месяца       11
13 лет и 2 месяца       11
8 лет и 6 месяцев       11
16 лет и 5 месяцев      11
13 лет и 4 месяца       11
7 лет и 8 месяцев       11
9 лет и 6 месяцев       11
15 лет и 5 месяцев      11
10 лет и 3 месяца       11
12 лет и 5 месяцев      11
6 лет и 6 месяцев       11
8 лет и 7 месяцев       10
9 лет и 8 месяцев       10
6 лет                   10
                        ..
27 лет и 5 месяцев       1
31 год и 7 месяцев       1
31 год и 6 месяцев       1
41 год и 3 месяца        1
39 лет и 3 месяца        1
26 лет и 7 месяцев       1
3

In [39]:
df = df.dropna(subset=['experience_length'])

In [40]:
df.shape

(1918, 12)

In [41]:
df.index = np.arange(df.shape[0])

In [42]:
df.columns

Index(['age', 'education', 'experience', 'experience_length', 'gender',
       'languages', 'name', 'salary', 'skills', 'city_Msk', 'city_Spb',
       'city_oth'],
      dtype='object')

In [43]:
df.head(3)

,age,education,experience,experience_length,gender,languages,name,salary,skills,city_Msk,city_Spb,city_oth
0,39,"[{'kind': 'Высшее образование', 'form': 'Дневн...","[{'period': 'октябрь 2017 – работает сейчас', ...",18 лет,0,NaN,"Администратор, управляющий, IT специалист",50 000₽,[{'professional': '• 1С: Предприятие • 1С: Роз...,1,0,0
1,47,"[{'kind': 'Высшее образование', 'form': 'Вечер...","[{'period': 'февраль 2008 – работает сейчас', ...",12 лет и 10 месяцев,0,"['Английский язык, Технический']",Администратор / Инженер технической поддержки IT,30 000₽,[{'professional': 'Английский язык уровень. Pr...,0,0,1
2,23,"[{'kind': 'Высшее образование', 'form': 'Дневн...","[{'period': 'июль 2015 – работает сейчас', 'le...",4 года и 3 месяца,1,NaN,"Администратор, IT-специалист, бизнес-аналитик,...",25 000₽,"[{'professional': 'Веб-дизайн, photoshop, MS O...",0,0,1


### Промежуточный итог: ###

Остались пропуски в 'languages', нужно решить, что с ними делать. Идея в том, что NaN можно интерпретировать как незнание языков. 

В остальных фичах пропусков нет. 

Город и пол one-hot.

Что с образованием - неясно, очень много категорий.

Для 'experience' и 'skills' нужно посмотреть распределения N-грамм и выбрать хорошие, информативные фичи по вхождениям ключевых слов.

Так же необходимо все числовые признаки нормализовать.

In [44]:
def period_to_month(string):
    s = string.split()
    if len(s) == 2:
        if (s[1] == 'год') or (s[1] == 'года') or (s[1] == 'лет'):
            return int(s[0]) * 12
        return int(s[0])
    return int(s[0]) * 12 + int(s[3])

In [45]:
pers = df.experience_length.copy()

In [46]:
pers is df.experience_length

False

In [47]:
for i in pers.index:
    pers.values[i] = period_to_month(pers.values[i])

In [48]:
df.experience_length = pers
df.experience_length

0       216
1       154
2        51
3        14
4        28
5        55
6        51
7        59
8       224
9         3
10       78
11      172
12      361
13      411
14      111
15      250
16      183
17      284
18      216
19      280
20      381
21      186
22      126
23      128
24       42
25      268
26      167
27      334
28      330
29      346
       ... 
1888    136
1889    121
1890    283
1891    283
1892    207
1893    244
1894    167
1895    121
1896    179
1897    165
1898    389
1899    334
1900    245
1901     97
1902    194
1903    211
1904    154
1905    268
1906     75
1907    298
1908    156
1909    194
1910    339
1911    156
1912    146
1913     66
1914    180
1915    347
1916     34
1917     72
Name: experience_length, Length: 1918, dtype: object

Перевели стаж работы в месяцы.

add more text

In [ ]:
df